# one-year-industrial-component-degradation
Alumno: Ibarra Medrano Jose Pablo
Universidad: UNISON
Dataset:  https://www.kaggle.com/inIT-OWL/one-year-industrial-component-degradation

**Importamos las librerias necesarias**

In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.svm import OneClassSVM
from numpy.random import seed
from keras.layers import Input, Dropout
from keras.layers.core import Dense 
from keras.models import Model, Sequential, load_model
from keras import regularizers
from keras.models import model_from_json
from scipy.special import softmax

# Contexto
##### Este dataset contiene la informacion de un compomente en degradacion durante 12 meses y se registraron sus cambios. Y se inició en el proyecto europeo de investigación e innovación IMPROVE.

**Primero pasamos el df a un variable llamada main_df el cual contienes todos los archivos de los 12 meses guardados en un solo csv para manejar mas facilmente los datos**

In [2]:
main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')
main_df.describe()

**Ahora busquemos la correlación entre los datos. Como se puede ver, lo único que parece correlacionase es el motor torque, la velocidad de la hoja y el error de retraso de la hoja, también la velocidad VAX y la velocidad de la envolvedora. Excluyendo, por supuesto, el mes y el número de muestra, y todas las autocorrelaciones en la diagonal.**

In [3]:
#heatmap e correlaciones de -1 a 1
sns.heatmap(main_df.corr(), vmin= -1, vmax = 1)

**Eliminemos algunas columnas y volteemos nuestros valores de columna de motor torque multiplicándo por -1. Todo esto para una mayor comprensión visual de la grafica**

In [4]:
main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')
#Eliminando columnas sin sentido para esta propuesta                           (axis=1) = columns
main_df = main_df.drop(['day', 'hour', 'sample_Number', 'month', 'timestamp'], axis=1)
#Volteando los valores de la columna
main_df['pCut::Motor_Torque'] = main_df['pCut::Motor_Torque'] *-1
#Heatmap
sns.heatmap(main_df.corr(), vmin= -1, vmax = 1)


## Tratamiento de datos no numéricos

**En nuestro conjunto de datos, los modos de la máquina pueden influir en los patrones de datos, por lo que necesitamos transformar esos datos de cadenas a clases numéricas. Esta función hará precisamente eso.** 

In [5]:
def handle_non_numeric(df):
    # Tomamos los valores de las columnas de nuestro df
    columns = df.columns.values
    
    for column in columns:
        
        # Diccionario con cada valor numerico con cada texto
        text_digit_vals = {}
        
        # Receibimos texto para convertilo a valor numerico
        def convert_to_int (val):
            

            return text_digit_vals[val]
        
        # Verificamos que los valores no sean flotantes o enteros
        if df[column].dtype !=np.int64 and df[column].dtype != np.float64:
            
            # Obtenemos los valores de la columna actual
            column_contents = df[column].values.tolist()
            
            # Obtenemos los valores unicos de la columna
            unique_elements = set(column_contents)
            x=0
            
            for unique in unique_elements:
                
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
            
            df[column] = list(map(convert_to_int, df[column]))
    
    return df

# Approach

**Se probarán algunos algoritmos para ver si podemos obtener información sobre la máquina. Usaremos OneClass SVM y KMeans con 1 clúster para probar la agrupación. Después de eso, probaremos un Autoencoder para intentar reproducir datos basados en el estado de salud de las máquinas.**

**En los 3 casos, tomaremos un segmento de las primeras filas y lo consideraremos como el estado correcto de la máquina, luego lo enviaremos a los algoritmos. Después de eso, le daremos a los algoritmos la totalidad del conjunto de datos y veremos cómo funcionan en el resto de los datos. Las desviaciones, puntuaciones bajas y pérdidas elevadas se considerarán anomalías a estudiar.**

# OneClass SVM approach

**OneClass SVM se utiliza para la detección de valores atípicos, intenta encontrar 2 clases en los datos, la clase "normal" y los valores atípicos. Usaremos la SVM para intentar encontrar valores atípicos y anomalías.**

In [ ]:
#Tomando todo el conjunto de datos
main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')
#Eliminando columnas con información no deseada / irrelevante para el algoritmo
main_df = main_df.drop(['day', 'hour', 'sample_Number', 'month', 'timestamp'], axis=1)
#Transformando modos en datos clasificados
main_df = handle_non_numeric(main_df)

X = main_df

#Definiendo el procesamiento para los datos
scaler = preprocessing.MinMaxScaler()
#Preprocesamiento
X = pd.DataFrame(scaler.fit_transform(X), 
                              columns=X.columns, 
                              index=X.index)


#Escalando
X = preprocessing.scale(X)
#Obtenemos las primeras 200.000 filas.
X_train = X[:200000]


#Creación de una SVM OneClass adecuada
ocsvm = OneClassSVM(nu=0.25, gamma=0.05)
ocsvm.fit(X_train)

**Predecir y clasificar el conjunto de datos en anomalías y no anomalías, luego pasarlo a un marco de datos.**

In [ ]:
df=main_df.copy()
df['anomaly'] = pd.Series(ocsvm.predict(X))

**Guardando el Dataframe**

In [ ]:
#Saving Dataframe.
df.to_csv('Labled_df.csv')

**Leyendo el dataframe**

In [ ]:
#Reading into dataframe
df = pd.read_csv('../input/created/Labled_df.csv', index_col=0)
df.head()

**Visualizando anomalias**

In [ ]:
#Obteniendo grupos etiquetados
scat_1 = df.groupby('anomaly').get_group(1)
scat_0 = df.groupby('anomaly').get_group(-1)

# Plot size
plt.subplots(figsize=(15,7))

# Plot group 1 -labeled, color green, point size 1
plt.plot(scat_1.index,scat_1['pCut::Motor_Torque'], 'g.', markersize=1)

# Plot group -1 -labeled, color red, point size 1
plt.plot(scat_0.index, scat_0['pCut::Motor_Torque'],'r.', markersize=1)


**Visualizando puntuaciones para todo el conjunto de datos**

In [ ]:
#Crear un marco de datos para la puntuación de cada muestra de datos
score = pd.DataFrame()
#Devolviendo puntajes al df
score['score'] = ocsvm.score_samples(X)

#Plot size
plt.subplots(figsize=(15,7))
#Plotting
score['score'].plot()
#Guardando puntajes dataframe
score.to_csv('SVM_Score.csv')

**Media móvil de puntuación invertida**

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))


((score['score'].rolling(20000).mean())*-1).plot(ax=ax)

**Scatplot para ver la partitura a través del ruido**

In [ ]:
plt.subplots(figsize=(15,7))
plt.plot(score.index, score['score'],'r.', markersize=1)

# KMeans approach

**El enfoque de Kmeans hará lo mismo que el OC-SVM**

In [ ]:
#------ Preparando funciones para entrenamiento y predicción futura -----
main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')
main_df = main_df.drop(['day', 'hour', 'sample_Number', 'month', 'timestamp'], axis=1)
main_df = handle_non_numeric(main_df)
X = main_df

scaler = preprocessing.MinMaxScaler()

X = pd.DataFrame(scaler.fit_transform(X), 
                              columns=X.columns, 
                              index=X.index)

X = preprocessing.scale(X)
#-------------------------------------------------------------------


#Porcentaje de los datos que se considerarán condición saludable
train_percentage = 0.15
#Valor entero para el segmento que se considerará en buen estado
train_size = int(len(main_df.index)*train_percentage)
#Tomando parte de los datos de entrenamiento
X_train = X[:train_size]


kmeans = KMeans(n_clusters=1)

kmeans.fit(X_train)

k_anomaly = main_df.copy()

k_anomaly = pd.DataFrame(kmeans.transform(X))

k_anomaly.to_csv('KM_Distance.csv')

plt.subplots(figsize=(15,7))

plt.plot(k_anomaly.index, k_anomaly[0], 'g', markersize=1)

# AutoEncoder approach

**AutoEncoders son redes neuronales que expanden y comprimen datos en dimensiones más altas y más bajas, luego intenta recrear los datos. La idea es que el autocodificador comprenderá la relación entre las características y, a partir de eso, recreará los datos exactos que se le dieron.**

**Entrenaremos el algoritmo con el estado saludable de la máquina. A medida que intenta reconstruir el resto de los datos como el estado correcto, la pérdida de reconstrucción, la diferencia entre los datos de la máquina pronosticados y los datos de la máquina real, se considerará un estado "incorrecto"**

In [ ]:
#------------------------- Preparando informacion para el entrenamiento --------------------------- 
main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')
main_df = main_df.drop(['day', 'hour', 'sample_Number', 'month', 'timestamp'], axis=1)
main_df = handle_non_numeric(main_df)
X = main_df

scaler = preprocessing.MinMaxScaler()

X = pd.DataFrame(scaler.fit_transform(X), 
                              columns=X.columns, 
                              index=X.index)



X = preprocessing.scale(X)


train_percentage = 0.15
train_size = int(len(main_df.index)*train_percentage)

X_train = X[:train_size]
#----------------------------------------------------------------------------------



#Semilla para validación y entrenamiento de lotes aleatorios
seed(10)
act_func = 'elu'
model=Sequential()

# Primera capa oculta, conectada al vector de entrada X.
model.add(Dense(50,activation=act_func,
                kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.0),
                input_shape=(X_train.shape[1],)
               )
         )
# Segunda capa oculta
model.add(Dense(10,activation=act_func,
                kernel_initializer='glorot_uniform'))
# Tercera capa oculta
model.add(Dense(50,activation=act_func,
                kernel_initializer='glorot_uniform'))

# Capa de entrada
model.add(Dense(X_train.shape[1],
                kernel_initializer='glorot_uniform'))

# Función de pérdida y elección del optimizador
model.compile(loss='mse',optimizer='adam')

# Modelo de tren para 50 épocas, tamaño de lote de 200
NUM_EPOCHS=50
BATCH_SIZE=200

#Aprovechando la validación y la pérdida de entrenamiento durante épocas
history=model.fit(np.array(X_train),np.array(X_train),
                  batch_size=BATCH_SIZE, 
                  epochs=NUM_EPOCHS,
                  validation_split=0.1,
                  verbose = 1)


**Graficar la pérdida de validación y la pérdida de entrenamiento a lo largo de los epochs**

In [ ]:
plt.subplots(figsize=(15,7))

plt.plot(history.history['loss'],'b',label='Training loss')
plt.plot(history.history['val_loss'],'r',label='Validation loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss, [mse]')

plt.show()

**Ahora alimentaremos el algoritmo con los mismos datos de entrenamiento y haremos que intente reconstruir los datos. Luego veremos la distribución de la pérdida sobre los datos de entrenamiento, más adelante usaremos esta distribución para determinar algunos umbrales.**

In [ ]:
#Reconstrucción de datos de entrenamiento
X_pred = model.predict(np.array(X_train))
X_pred = pd.DataFrame(X_pred,columns=main_df.columns)
X_pred.index = pd.DataFrame(X_train).index

scored = pd.DataFrame(index=pd.DataFrame(X_train).index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_train), axis = 1)

plt.subplots(figsize=(15,7))
sns.distplot(scored['Loss_mae'],
             bins = 15, 
             kde= True,
            color = 'blue');





**Ahora para hacer lo mismo pero con todos nuestros datos para ver la pérdida a lo largo del tiempo, esto nos dará datos interesantes.**

In [ ]:

#Reconstruyendo toda la informacion
X_pred = model.predict(np.array(X))
X_pred = pd.DataFrame(X_pred,columns=main_df.columns)
X_pred.index = pd.DataFrame(X).index

#Devolviendo la media de las pérdidas para cada columna y poniéndola en un marco de datos
scored = pd.DataFrame(index=pd.DataFrame(X).index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X), axis = 1)

plt.subplots(figsize=(15,7))


#Guardando el dataframe
scored.to_csv('AutoEncoder_loss.csv')

plt.plot(scored['Loss_mae'],'b',label='Prediction Loss')

plt.legend(loc='upper right')
plt.xlabel('Sample')
plt.ylabel('Loss, [mse]')

# Resultados del analysis

### Diagrama de dispersión para la puntuación de cada algoritmo, para ver a través del ruido.

In [ ]:
plt.subplots(figsize=(15,7))

enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')
plt.plot(enc_loss.index,enc_loss['Loss_mae'], 'g.', markersize=1,label="AutoEncoder Loss")
plt.legend(loc='upper right')
plt.xlabel('Sample')
plt.show()

plt.subplots(figsize=(15,7))
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
plt.plot(k_anomaly.index,k_anomaly['0'], 'g.', markersize=1,label="KM cluster Distance")
plt.legend(loc='upper right')
plt.xlabel('Sample')
plt.show()

plt.subplots(figsize=(15,7))
score = pd.read_csv('../input/created/SVM_Score.csv')
plt.plot(score.index,score['score'], 'g.', markersize=1,label="OCSVM score")
plt.legend(loc='upper right')
plt.xlabel('Sample')
plt.show()


**Graficando cada algoritmo, con OCSVM invertido sobre 0.**

In [ ]:
#Plot size
plt.subplots(figsize=(15,7))

#Leyendo cada csv
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

#Escalando datos para la visualización
k_distance = k_anomaly/k_anomaly.max()
svm_score = (score/score.max())*-1

plt.plot(enc_loss.index,enc_loss['Loss_mae'], label="AutoEncoder Loss")
plt.plot(svm_score.index, svm_score['score'],label="OCSVM score")
plt.plot(k_distance.index,k_distance['0'], label="Kmeans Euclidean Dist")



plt.gca().legend(('AutoEncoder Loss','OCSVM score * -1','Kmeans Euclidean Dist'))


**Buscando correlación entre los algoritmos**

In [ ]:
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']
corr.corr()

**Comparando los tres metodos de deteccion de anomalia por medio de Scatter plots con media móvil**

In [ ]:
#---- Leyendo datos y pasarlos al marco de datos nuevamente ----- 

k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']
#---------------------------------------------------------



plt.subplots(figsize=(15,7))

#Scatter plot de SVM
plt.plot(corr.index, corr['SVM_score'], 'g.', markersize=1, label = 'OCSVM_score')
#Trazado de la media móvil de 1000
plt.plot(corr.index, corr['SVM_score'].rolling(1000).mean(), 'r', markersize=1, label = 'Moving Mean')
plt.legend(loc='upper right')
plt.show()


plt.subplots(figsize=(15,7))
plt.plot(corr.index, corr['KM_cluster_distance'], 'g.', markersize=1, label = 'KM_cluster_distance')
plt.plot(corr.index, corr['KM_cluster_distance'].rolling(1000).mean(), 'r', markersize=1, label = 'Moving Mean')
plt.legend(loc='upper right')
plt.show()


plt.subplots(figsize=(15,7))
plt.plot(corr.index, corr['AutoEnc_loss'], 'g.', markersize=1, label = 'AutoEnc_loss')
plt.plot(corr.index, corr['AutoEnc_loss'].rolling(1000).mean(), 'r', markersize=1, label = 'Moving Mean')
plt.legend(loc='upper right')
plt.show()



**Distribución de pérdidas sobre los datos de entrenamiento**

In [ ]:
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']



plt.subplots(figsize=(10,7))
sns.distplot(corr['SVM_score'].head(160000), bins=15)
plt.show()

plt.subplots(figsize=(10,7))
sns.distplot(corr['KM_cluster_distance'].head(160000),bins=15)
plt.show()

plt.subplots(figsize=(10,7))
sns.distplot(corr['AutoEnc_loss'].head(160000),bins=15)
plt.show()

**Distribución de pérdidas en todo el conjunto de datos**

In [ ]:
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']

plt.subplots(figsize=(10,7))
sns.distplot(corr['SVM_score'], bins=15)
plt.show()

plt.subplots(figsize=(10,7))
sns.distplot(corr['KM_cluster_distance'],bins=15)
plt.show()

plt.subplots(figsize=(10,7))
sns.distplot(corr['AutoEnc_loss'],bins=15)
plt.show()

**Ahora usaremos la información sobre la distribución de la pérdida de entrenamiento para determinar algunos umbrales para los gráficos. También se trazarán los medios móviles.**

In [ ]:
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']



#Creando una matriz para que los umbrales se tracen en todo el conjunto de datos
lower_threshold = np.full((corr['SVM_score'].size, 1), 0)
upper_threshold = np.full((corr['SVM_score'].size, 1), 18000)
high_density_threshold = np.full((corr['SVM_score'].size, 1), 13250)

plt.subplots(figsize=(15,7))
plt.plot(corr.index, corr['SVM_score'], 'k', markersize=1, label = 'OCSVM_score')
plt.plot(corr.index, corr['SVM_score'].rolling(100).mean(), 'r', markersize=1, label = 'Moving Mean')
plt.plot(corr.index, lower_threshold, label='Lower Threshold')
plt.plot(corr.index, upper_threshold, label = 'Upper Threshold')
plt.plot(corr.index, high_density_threshold, label = 'Highest Density')
plt.legend(loc='upper right')
plt.show()


lower_threshold = np.full((corr['KM_cluster_distance'].size, 1), 1.2)
upper_threshold = np.full((corr['KM_cluster_distance'].size, 1), 17.5)
high_density_threshold = np.full((corr['KM_cluster_distance'].size, 1), 2.5)

plt.subplots(figsize=(15,7))
  
plt.plot(corr.index, corr['KM_cluster_distance'], 'k', markersize=1, label = 'KM_cluster_distance')
plt.plot(corr.index, corr['KM_cluster_distance'].rolling(100).mean(), 'r', markersize=1, label = 'Moving Mean')
plt.plot(corr.index, lower_threshold, label='Lower Threshold')
plt.plot(corr.index, upper_threshold, label = 'Upper Threshold')
plt.plot(corr.index, high_density_threshold, label = 'Highest Density')
plt.legend(loc='upper right')
plt.show()

lower_threshold = np.full((corr['AutoEnc_loss'].size, 1), 0)
upper_threshold = np.full((corr['AutoEnc_loss'].size, 1), 0.1)
high_density_threshold = np.full((corr['AutoEnc_loss'].size, 1), 0.05)

plt.subplots(figsize=(15,7))
  
plt.plot(corr.index, corr['AutoEnc_loss'], 'k', markersize=1, label = 'AutoEnc_loss')
plt.plot(corr.index, corr['AutoEnc_loss'].rolling(100).mean(), 'r', markersize=1, label = 'Moving Mean')
plt.plot(corr.index, lower_threshold, label='Lower Threshold')
plt.plot(corr.index, upper_threshold, label = 'Upper Threshold')
plt.plot(corr.index, high_density_threshold, label = 'Highest Density')
plt.legend(loc='upper right')
plt.show()


**Scatter plot de los puntajes del algoritmo vs los otros algoritmos**

In [ ]:
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']

plt.subplots(figsize=(15,7))
  
plt.plot(corr['KM_cluster_distance'],corr['SVM_score'],'b.',markersize=1 )
plt.xlabel('KM')
plt.ylabel('SVM')
plt.show()

plt.subplots(figsize=(15,7))
  
plt.plot(corr['AutoEnc_loss'],corr['SVM_score'],'b.' ,markersize=1 )
plt.xlabel('Encoder')
plt.ylabel('SVM')
plt.show()

plt.subplots(figsize=(15,7))
  
plt.plot(corr['AutoEnc_loss'],corr['KM_cluster_distance'],'b.' ,markersize=1 )
plt.xlabel('Encoder')
plt.ylabel('KM')
plt.show()

# Continuing with Autoencoder
**Aunque existe cierta simularidad de anomalías visuales entre los algoritmos, los algoritmos de agrupamiento nos dan mucho ruido y poco en lo que trabajar. Por otro lado, el codificador automático tiene una carrera casi segura hasta el punto de falla. No podemos concluir con absoluta certeza que los componentes hayan cambiado después del pico de pérdida más alto, pero es muy posible.**

**Además, ahora analizaremos la pérdida del codificador por mes, con los umbrales.**  

In [ ]:
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']

main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')

#Pasar la pérdida del codificador al marco de datos principal, para que sea más fácil separar por mes
main_df['AutoEnc_loss'] = corr['AutoEnc_loss']

#Obteniendo lista de los meses
months = main_df['month'].dropna().unique()

#Recorriendo cada mes
for month in months:
    #Tomando la porción del marco de datos para cada mes
    month_df = main_df.groupby('month').get_group(month)
    
    upper_threshold = np.full((month_df['AutoEnc_loss'].size, 1), 0.1)
    high_density_threshold = np.full((month_df['AutoEnc_loss'].size, 1), 0.05)
    
    plt.subplots(figsize=(15,7))
    plt.plot(month_df.index, month_df['AutoEnc_loss'], label=f'AutoEnc_loss month_{month}')
    plt.plot(month_df.index, upper_threshold, label = 'Upper Threshold')
    plt.plot(month_df.index, high_density_threshold, label = 'Highest Density')
    plt.legend(loc='upper right')
    plt.ylim(0,1.3)
    
    plt.show()
    
    

**Ahora veremos la distribución de pérdidas por mes.**

In [ ]:
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']

main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')

main_df['AutoEnc_loss'] = corr['AutoEnc_loss']

months = main_df['month'].dropna().unique()

for month in months:
    month_df = main_df.groupby('month').get_group(month)    
    
    plt.subplots(figsize=(15,7))
    sns.distplot((month_df['AutoEnc_loss']), bins=15).set_title(f'Month {month} Loss Distribution')
    #X axis limits
    plt.xlim([-1.2,1.2])
    plt.show()

**Una forma interesante de ver la anomalía es la cola de la distribución. Como pudimos ver en la gráfica por mes, el cuarto mes tiene el punto de anomalía más alto. También es evidente la influencia de eso en la distribución de la pérdida. Quizás al observar la curtosis de cada mes podamos obtener más información.**

**La curtosis nos informará sobre la forma de la distribución. Una curtosis alta significa que muchos puntos de datos tienen el mismo valor y que las colas o la desviación estándar son realmente pequeñas o inexistentes (en nuestro caso, muchos puntos de datos cercanos a 0 significan una buena condición de la máquina). Curtosis baja significa que tenemos muchos puntos de datos dispersos, lo que le da a la distribución colas anchas y gruesas, casi del tamaño de su pico.** 

In [ ]:
k_anomaly = pd.read_csv('../input/created/KM_Distance.csv')
score = pd.read_csv('../input/created/SVM_Score.csv')
enc_loss = pd.read_csv('../input/created/AutoEncoder_loss.csv')

corr = pd.DataFrame()
corr['SVM_score'] = score['score']
corr['KM_cluster_distance'] = k_anomaly['0']
corr['AutoEnc_loss'] = enc_loss['Loss_mae']

main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')

main_df['AutoEnc_loss'] = corr['AutoEnc_loss']

months = main_df['month'].dropna().unique()

for month in months:
    month_df = main_df.groupby('month').get_group(month)
    kurt = (month_df['AutoEnc_loss']).kurtosis()
    print(f'Month {month} kurtosis = {kurt}')

**Así que los meses con baja curtosis son los meses con más anomalías, lo que nos puede decir un poco sobre el estado de la máquina.**

# Detección de sensor

**Ahora que sabemos dónde tiene un problema la máquina, intentaremos encontrar qué componente / sensor está causando esta perturbación en el codificador automático. Para eso lo entrenaremos nuevamente, y obtendremos sus predicciones y pérdidas para cada columna para ver cuál tiene la mayor contribución a la pérdida total.**

In [ ]:
main_df = pd.read_csv('../input/datasetsone-year-compiledcsv/One_year_compiled.csv')
main_df = main_df.drop(['day', 'hour', 'sample_Number', 'month', 'timestamp'], axis=1)
main_df = handle_non_numeric(main_df)
X = main_df

scaler = preprocessing.MinMaxScaler()

X = pd.DataFrame(scaler.fit_transform(X), 
                              columns=X.columns, 
                              index=X.index)
X = preprocessing.scale(X)

train_percentage = 0.15
train_size = int(len(main_df.index)*train_percentage)

X_train = X[:train_size]

seed(10)

act_func = 'elu'

# Input layer:
model=Sequential()
# First hidden layer, connected to input vector X. 
model.add(Dense(50,activation=act_func,
                kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.0),
                input_shape=(X_train.shape[1],)
               )
         )

model.add(Dense(10,activation=act_func,
                kernel_initializer='glorot_uniform'))

model.add(Dense(50,activation=act_func,
                kernel_initializer='glorot_uniform'))

model.add(Dense(X_train.shape[1],
                kernel_initializer='glorot_uniform'))

model.compile(loss='mse',optimizer='adam')

# Train model for 100 epochs, batch size of 10: 
NUM_EPOCHS=50
BATCH_SIZE=200

history=model.fit(np.array(X_train),np.array(X_train),
                  batch_size=BATCH_SIZE, 
                  epochs=NUM_EPOCHS,
                  validation_split=0.1,
                  verbose = 1)

In [ ]:
#Prediciendo y pasando la predicción al dataframe
X_pred = model.predict(np.array(X))
X_pred = pd.DataFrame(X_pred,columns=main_df.columns)
X_pred.index = pd.DataFrame(main_df).index

#Pasando X de un array a un dataframe
X = pd.DataFrame(X,columns=main_df.columns)
X.index = pd.DataFrame(main_df).index

#Dataframe donde irá toda la pérdida por columnas
loss_df = pd.DataFrame()

main_df.drop('mode',axis=1, inplace=True)

for column in main_df.columns:
    loss_df[f'{column}'] = (X_pred[f'{column}'] - X[f'{column}']).abs()
     
    plt.subplots(figsize=(15,7))
    plt.plot(loss_df.index, loss_df[f'{column}'], label=f'{column} loss')
    plt.legend(loc='upper right')
    
    plt.show()

loss_df.to_csv('AutoEncoder_loss_p_column.csv')

**Ahora aplicaremos la función Softmax a cada fila para que podamos obtener el porcentaje que cada columna contribuye a la pérdida total. En cuanto a la suma de cada fila nos dará 1.**

In [ ]:
sftmax_df = pd.read_csv('../input/created/AutoEncoder_loss_p_column.csv', index_col=0)
sftmax_df = softmax(sftmax_df, axis=1)
sftmax_df.describe()

**Trazando el porcentaje de la contribución de cada columna a la pérdida total**

In [ ]:
for column in sftmax_df.columns:
    

    plt.subplots(figsize=(15,7))
    plt.plot(sftmax_df.index, sftmax_df[f'{column}'], label=f'{column} loss')
    plt.legend(loc='upper right')
    
    plt.show()

**Ahora podemos trazar un diagrama de pila para visualizar mejor la contribución de cada columna a la pérdida total. Como verá, la posición de Blades contribuye mucho a la pérdida total en ese pico que vimos. Miraremos más de cerca a esa porción.**

In [ ]:
plt.subplots(figsize=(15,7))

#Labels for stackbar plot
df_label = ['Torque', 'Cut lag','Cut speed','Cut position','Film position','Film speed','Film lag','VAX']

#Stackbar plot
plt.stackplot(sftmax_df.index, sftmax_df['pCut::Motor_Torque'],
             sftmax_df['pCut::CTRL_Position_controller::Lag_error'],
             sftmax_df['pCut::CTRL_Position_controller::Actual_speed'],
              sftmax_df['pCut::CTRL_Position_controller::Actual_position'],
             sftmax_df['pSvolFilm::CTRL_Position_controller::Actual_position'],
             sftmax_df['pSvolFilm::CTRL_Position_controller::Actual_speed'],
             sftmax_df['pSvolFilm::CTRL_Position_controller::Lag_error'],
             sftmax_df['pSpintor::VAX_speed'],
             labels = df_label)

plt.legend(loc='upper center', ncol=8)

plt.ylim(0,1)

In [ ]:
plt.subplots(figsize=(15,7))

df_label = ['Torque', 'Cut lag','Cut speed','Cut position','Film position','Film speed','Film lag','VAX']

sftmax_df = sftmax_df[400000:600000]

plt.stackplot(sftmax_df.index, sftmax_df['pCut::Motor_Torque'],
             sftmax_df['pCut::CTRL_Position_controller::Lag_error'],
             sftmax_df['pCut::CTRL_Position_controller::Actual_speed'],
              sftmax_df['pCut::CTRL_Position_controller::Actual_position'],
             sftmax_df['pSvolFilm::CTRL_Position_controller::Actual_position'],
             sftmax_df['pSvolFilm::CTRL_Position_controller::Actual_speed'],
             sftmax_df['pSvolFilm::CTRL_Position_controller::Lag_error'],
             sftmax_df['pSpintor::VAX_speed'],
             labels = df_label)

plt.legend(loc='upper center', ncol=8)

plt.ylim(0,1)

**Parece que el error de retraso de la hoja también proporciona una gran parte de la pérdida total. La posible explicación aquí es que la cuchilla está desgastada y, por eso, está comenzando a desviarse del camino que la máquina intenta trazar para la cuchilla al cortar la película.**

**Ahora veremos la distribución de la contribución de cada columna a la pérdida total, solo para comprender mejor qué sensores están dando mayor pérdida.**

In [ ]:
for column in sftmax_df.columns:
    

    plt.subplots(figsize=(15,7))
    
    sns.distplot(( sftmax_df[f'{column}']), bins=15).set_title(f'Contribution Distribution')
    plt.xlim(0,1)
    
    plt.show()

# Conclusion 

**Con toda la información recopilada, podríamos decir dónde y cuándo la máquina sufrió una degradación masiva. También pudimos decir cuál de las medidas contribuye más a la pérdida en todo el año de la máquina, lo que nos dice que estos componentes fabricados necesitan más atención. Se pueden utilizar umbrales de percentiles, análisis de anomalías de distribución, SVM y muchos otros métodos para la detección del desgaste de los componentes a lo largo del tiempo con la información que se proporciona aquí.**

**Un problema con este método es la necesidad de preprocesar y escalar los datos por completo para luego entregarlos al algoritmo. Una forma de superar este problema es utilizar este conjunto de datos o una porción de él y combinarlo con una nueva porción de datos para los análisis del estado del sistema. Y para cada nuevo segmento de datos que necesite analizar, eliminamos el segmento agregado anterior y mantenemos este conjunto de datos intacto.** 

# References

**-- https://www.sciencedirect.com/science/article/pii/S221282711830307X**

**-- https://towardsdatascience.com/machine-learning-for-anomaly-detection-and-condition-monitoring-d4614e7de770**

